# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
load_csv = "output_data/cities.csv"
city_df = pd.read_csv(load_csv)
city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Albany,42.60,-73.97,46.99,90,100,3.58,US,1970-01-01 00:00:01.603938562
1,1,Cape Town,-33.93,18.42,61.00,87,75,17.22,ZA,1970-01-01 00:00:01.603938537
2,2,Punta Arenas,-53.15,-70.92,46.40,65,40,34.45,CL,1970-01-01 00:00:01.603938562
3,3,Tura,25.52,90.22,80.22,73,0,1.10,IN,1970-01-01 00:00:01.603938626
4,4,Jamestown,42.10,-79.24,43.00,86,1,8.05,US,1970-01-01 00:00:01.603938572


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Lat and Lng as locations and Humidity as the weight
locations = city_df[["Lat", "Lng"]]
humidity = city_df['Humidity'].astype(int)


# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Narrow down the cities to fit weather conditions.

narrowed_city_df = city_df.copy()
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] >= 70]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] <= 80]  
narrowed_city_df = narrowed_city_df[narrowed_city_df['Wind Speed'] <= 10]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Cloudiness'] == 0]

#Drop any rows will null values
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df = narrowed_city_df.reset_index()
narrowed_city_df

,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,78,78,Santa Rosa de Viterbo,-21.47,-47.36,77.00,53,0,6.93,BR,1970-01-01 00:00:01.603938916
1,121,121,São Filipe,14.90,-24.50,79.39,80,0,1.97,CV,1970-01-01 00:00:01.603938562
2,130,130,Birao,10.28,22.79,71.67,43,0,2.75,CF,1970-01-01 00:00:01.603938979
3,181,181,Salalah,17.02,54.09,78.80,74,0,3.36,OM,1970-01-01 00:00:01.603938962
4,186,186,Katsuura,35.13,140.30,72.00,68,0,8.05,JP,1970-01-01 00:00:01.603939047
5,190,190,Bandarbeyla,9.49,50.81,77.65,82,0,2.66,SO,1970-01-01 00:00:01.603939053
6,222,222,Nangal,31.37,76.38,70.27,24,0,2.93,IN,1970-01-01 00:00:01.603939090
7,226,226,Cape Coast,5.11,-1.25,76.57,90,0,5.53,GH,1970-01-01 00:00:01.603939117
8,241,241,Pūndri,29.75,76.55,71.20,29,0,2.93,IN,1970-01-01 00:00:01.603939138
9,242,242,Kaya,13.09,-1.08,75.40,18,0,4.99,BF,1970-01-01 00:00:01.603939139


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Store into variable named hotel_df
hotel_df = narrowed_city_df
hotel_df["Hotel Name"] = np.nan
hotel_df = hotel_df.drop(columns='index')
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,78,Santa Rosa de Viterbo,-21.47,-47.36,77.00,53,0,6.93,BR,1970-01-01 00:00:01.603938916,NaN
1,121,São Filipe,14.90,-24.50,79.39,80,0,1.97,CV,1970-01-01 00:00:01.603938562,NaN
2,130,Birao,10.28,22.79,71.67,43,0,2.75,CF,1970-01-01 00:00:01.603938979,NaN
3,181,Salalah,17.02,54.09,78.80,74,0,3.36,OM,1970-01-01 00:00:01.603938962,NaN
4,186,Katsuura,35.13,140.30,72.00,68,0,8.05,JP,1970-01-01 00:00:01.603939047,NaN
5,190,Bandarbeyla,9.49,50.81,77.65,82,0,2.66,SO,1970-01-01 00:00:01.603939053,NaN
6,222,Nangal,31.37,76.38,70.27,24,0,2.93,IN,1970-01-01 00:00:01.603939090,NaN
7,226,Cape Coast,5.11,-1.25,76.57,90,0,5.53,GH,1970-01-01 00:00:01.603939117,NaN
8,241,Pūndri,29.75,76.55,71.20,29,0,2.93,IN,1970-01-01 00:00:01.603939138,NaN
9,242,Kaya,13.09,-1.08,75.40,18,0,4.99,BF,1970-01-01 00:00:01.603939139,NaN


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
